# Preparing the walmart data. 

AIM: The notebook helps download walmart data and turn it to a format similar to M&S hierarchical data. [Link to the dataset](https://www.kaggle.com/competitions/m5-forecasting-accuracy)

<br></br>
<div style="text-align: center; line-height: 5; padding-top: 20px;  padding-bottom: 20px;">
  <img src="https://raw.githubusercontent.com/puneet-jain159/Image_dump/2a8b03eef9bd111b98b261846d57b72ce98fde38/walmart_data_description.png" alt='Push compute' height="1000" width="1600">
</div>

### Install Dependencies

In [ ]:
import pandas as pd
import numpy as np

### Clone Data from Kaggle

Below is shell script to download the data from kaggle .</br> You will need to sign in and generate access key to download the data and also agree to the data disclaimer other will hit a 403 error</br> After downloading to the local disk the data is copied to dbfs
```
kaggle competitions download -c m5-forecasting-accuracy -p /dbfs/walmart/data/
```

In [ ]:
%sh 
pip install kaggle
export KAGGLE_USERNAME=XXXXX
export KAGGLE_KEY=XXXXXXX
mkdir -p /dbfs/walmart/data/
kaggle competitions download -c m5-forecasting-accuracy -p /dbfs/walmart/data/
cd /dbfs/walmart/data/ &&  unzip /dbfs/walmart/data/m5-forecasting-accuracy.zip 

In [ ]:
dbutils.fs.ls("dbfs:/walmart/data/")

### Read the downloaded data and analyize all the different files

In [ ]:
sdf_calendar = spark.read.csv('dbfs:/walmart/data/calendar.csv', header="True")
sdf_calendar.cache()
print(sdf_calendar.count())
sdf_calendar.display()

In [ ]:
sdf_sales_train_evaluation = spark.read.csv('dbfs:/walmart/data/sales_train_evaluation.csv', header="True")
sdf_sales_train_evaluation.cache()
print( sdf_sales_train_evaluation.count())
sdf_sales_train_evaluation.display()

In [ ]:
sdf_sell_prices= spark.read.csv('dbfs:/walmart/data/sell_prices.csv', header="True")
sdf_sell_prices.cache()
print( sdf_sell_prices.count())
sdf_sell_prices.display()

### Transformation and clean the data
We transform and melt the column into rows

In [ ]:
ids =['id','item_id','dept_id','cat_id','store_id','state_id'] 
cols = [col for col in sdf_sales_train_evaluation.columns if col not in ids ] 

# pivot the data
sdf_sales_train_evaluation_pivot = (sdf_sales_train_evaluation.melt(
    ids=ids, values=cols,
    variableColumnName="date", valueColumnName="sale_quantity"))

sdf_sales_train_evaluation_pivot.write.mode("overwrite").format('delta').save('dbfs:/walmart/data/clean_data/sdf_sales_train_evaluation/')

In [ ]:
# sdf_sell_prices.filter((sdf_sell_prices.item_id == 'HOUSEHOLD_1_335') & (sdf_sell_prices.wm_yr_wk == '11105') & (sdf_sell_prices.store_id == 'CA_2')).display()

In [ ]:
# merge to get date 
sdf_calendar = sdf_calendar.withColumnRenamed("date", 'date_time')
cond = [sdf_sales_train_evaluation_pivot.date == sdf_calendar.d]
sdf_sales_train_evaluation_pivot = sdf_sales_train_evaluation_pivot.join(sdf_calendar.select(['d','wm_yr_wk','date_time']),cond, 'left')

# merge to get sales price
cond = [sdf_sales_train_evaluation_pivot.wm_yr_wk == sdf_sell_prices.wm_yr_wk,
        sdf_sales_train_evaluation_pivot.item_id == sdf_sell_prices.item_id,
        sdf_sales_train_evaluation_pivot.store_id == sdf_sell_prices.store_id ]
final_table = sdf_sales_train_evaluation_pivot.join(sdf_sell_prices,cond, 'left'). \
      select(sdf_sales_train_evaluation_pivot.item_id,sdf_sales_train_evaluation_pivot.dept_id,
             sdf_sales_train_evaluation_pivot.cat_id,sdf_sales_train_evaluation_pivot.store_id,
             sdf_sales_train_evaluation_pivot.state_id ,sdf_sales_train_evaluation_pivot.date_time,
             sdf_sell_prices.sell_price ,sdf_sales_train_evaluation_pivot.sale_quantity)

# write out the final table
final_table.write.mode("overwrite").format('delta').save('dbfs:/walmart/data/clean_data/final_cleaned_table/')

In [ ]:
# Clean the final table 

# remove nulls sell_price and sales_quantity = 0 
final_table = final_table.filter(final_table.sale_quantity > 0)
display(final_table)

item_id,dept_id,cat_id,store_id,state_id,date_time,sell_price,sale_quantity
FOODS_1_005,FOODS_1,FOODS,TX_1,TX,2011-01-29,2.94,2
FOODS_1_005,FOODS_1,FOODS,TX_1,TX,2011-01-30,2.94,3
FOODS_1_005,FOODS_1,FOODS,TX_1,TX,2011-01-31,2.94,1
FOODS_1_005,FOODS_1,FOODS,TX_1,TX,2011-02-02,2.94,2
FOODS_1_005,FOODS_1,FOODS,TX_1,TX,2011-02-04,2.94,1
FOODS_2_157,FOODS_2,FOODS,CA_1,CA,2011-01-29,3.25,1
FOODS_2_157,FOODS_2,FOODS,CA_1,CA,2011-01-30,3.25,1
FOODS_2_157,FOODS_2,FOODS,CA_1,CA,2011-02-01,3.25,3
FOODS_3_449,FOODS_3,FOODS,CA_4,CA,2011-01-29,1.98,5
FOODS_3_449,FOODS_3,FOODS,CA_4,CA,2011-01-30,1.98,3


### Write out the final table as a Delta Table

In [ ]:
#
final_table.write.mode("overwrite").format('delta').save('dbfs:/walmart/data/clean_data/final_cleaned_filtered')
